### Yaml 파일 생성

In [ ]:
import os
import pandas as pd
import ast

# CSV 파일 로드
csv_file = "output.csv"  # CSV 파일 경로 입력
output_label_dir = "dataset/labels/train"  # YOLO 라벨 저장 경로

# 클래스 매핑
classes = ["101_10", "101_20"]  # 클래스 정의
class_mapping = {cls: idx for idx, cls in enumerate(classes)}

# 출력 디렉토리 생성
os.makedirs(output_label_dir, exist_ok=True)

# CSV 데이터 읽기
data = pd.read_csv(csv_file)

# YOLO 형식으로 변환
for _, row in data.iterrows():
    full_name = row["FullName"] + ".jpg"  # 이미지 파일 이름
    bbox_data = row["BBox"]
    # print(bbox_data)

    # BBox 파싱
    try:
        # 문자열을 리스트로 변환
        if isinstance(bbox_data, str):
            bbox_list = ast.literal_eval(bbox_data)
        elif isinstance(bbox_data, list):
            bbox_list = list(bbox_data)
        else:
            bbox_list = []
    except (ValueError, SyntaxError):
        print(f"BBox 변환 오류: {bbox_data}")
        bbox_list = []

    # BBox가 비어있으면 건너뜁니다
    if not isinstance(bbox_list, list) or len(bbox_list) == 0:
        # print(f"빈 BBox: {row['FullName']}, 건너뜁니다.")
        continue

    # 클래스 ID 가져오기
    category = f"{row['Category']}_{row['Error']}"  # 클래스 이름 생성
    if category not in class_mapping:
        print(f"알 수 없는 클래스: {category}, 건너뜁니다.")
        continue

    class_id = class_mapping[category]

    # YOLO 라벨 파일 생성
    label_file = os.path.join(output_label_dir, full_name.replace(".jpg", ".txt"))
    with open(label_file, "w") as f:
            # BBox가 리스트인지 확인
            # if not isinstance(bbox, list) or len(bbox) != 4:
            #     # print(f"잘못된 BBox 형식: {bbox}, 건너뜁니다.")
            #     continue

        x_min, y_min, width, height = bbox_list
        x_center = x_min + (width / 2)
        y_center = y_min + (height / 2)
        print(x_min, y_min, width, height)

        # 이미지 크기 비율로 변환 (임의의 이미지 크기 사용, 실제 크기 반영 필요)
        img_width, img_height = 4000, 2000  # 예제 값, 실제 이미지 크기 사용
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height

            # YOLO 형식으로 저장
        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("YOLO 라벨 생성 완료!")

In [ ]:
pip install ultralytics

### 생성한 Yaml 파일로 train

In [49]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

model.train(data='/Users/shin-yeji/Downloads/162.부품 품질 검사 영상 데이터(자동차)/01.데이터/2.Validation/라벨링데이터/범퍼/스크래치/data.yaml' , epochs=5
            ,cache=False)

New https://pypi.org/project/ultralytics/8.3.64 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.9.6 torch-2.5.1 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/Users/shin-yeji/Downloads/162.부품 품질 검사 영상 데이터(자동차)/01.데이터/2.Validation/라벨링데이터/범퍼/스크래치/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=Fals

train: Scanning /Users/shin-yeji/Downloads/dataset/labels/train.cache... 4 images, 6 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]
val: Scanning /Users/shin-yeji/Downloads/dataset/labels/val.cache... 0 images, 3 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /Users/shin-yeji/Downloads/dataset/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/train12/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train12
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      3.237      23.75      2.432          5        640: 100%|██████████| 1/1 [00:05<00:00,  5.35s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]

...


5 epochs completed in 0.009 hours.
Optimizer stripped from runs/detect/train12/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train12/weights/best.pt, 22.5MB

Validating runs/detect/train12/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.9.6 torch-2.5.1 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]

                   all          3          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 0.8ms preprocess, 105.2ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to runs/detect/train12


ModuleNotFoundError: No module named 'numpy.char'